In [3]:
import json
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
!pip install seaborn
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable



- label: This feature indicates whether the file is labeled as benign (False) or malicious (True).
- .textsize: This feature represents the size of the .text section in the PE file.
- .textentropy: This feature represents the entropy of the .text section in the PE file.
- .textvsize: This feature represents the virtual size of the .text section in the PE file.
- .datasize: This feature represents the size of the .data section in the PE file.
- .dataentropy: This feature represents the entropy of the .data section in the PE file.
- .datavsize: This feature represents the virtual size of the .data section in the PE file.
- .rsrcsize: This feature represents the size of the .rsrc section in the PE file.
- .rsrcentropy: This feature represents the entropy of the .rsrc section in the PE file.
- .rsrcvsize: This feature represents the virtual size of the .rsrc section in the PE file.
- machine: This feature represents the type of machine for which the file is intended.
- subsystem: This feature represents the target subsystem for the PE file.
- generalsize: This feature represents the size of the general file information in the PE header.
- generalvsize: This feature represents the virtual size of the general file information in the PE header.
- generalhas_debug: This feature indicates whether the PE file has a debug section (True) or not (False).
- generalexports: This feature represents the number of exported functions in the PE file.
- generalimports: This feature represents the number of imported functions in the PE file.
- generalhas_relocations: This feature indicates whether the PE file has relocation information (True) or not (False).
- generalhas_resources: This feature indicates whether the PE file has resource information (True) or not (False).
- generalhas_signature: This feature indicates whether the PE file has a digital signature (True) or not (False).
- generalhas_tls: This feature indicates whether the PE file has thread local storage (TLS) information (True) or not (False).
- generalsymbols: This feature represents the number of symbols in the PE file.
- stringsnumstrings: This feature represents the number of printable strings in the PE file.
- stringsavlength: This feature represents the average length of the printable strings in the PE file.
- stringsprintabledist: This feature represents the histogram of printable characters within the printable strings.
- stringsprintables: This feature indicates whether the PE file contains printable strings (True) or not (False).
- stringsentropy: This feature represents the entropy of characters across all printable strings.
- stringspaths: This feature represents the number of strings that begin with "C:\" indicating paths.
- stringsurls: This feature represents the number of strings that contain "http://" or "https://" indicating URLs.
- stringsregistry: This feature represents the number of strings that contain "HKEY_" indicating registry keys.
- stringsMZ: This feature represents the number of strings that contain "MZ" indicating Windows PE droppers or bundled executables.
- cofftimestamp: This feature represents the timestamp in the COFF header of the PE file.
- coffmachine: This feature represents the target machine for which the PE file is intended.
- coffcharacteristics: This feature represents a list of image characteristics of the PE file.
- optionalsubsystem: This feature represents the target subsystem specified in the optional header of the PE file.
- optionaldll_characteristics: This feature represents a list of DLL characteristics of the PE file.
- optionalmagic: This feature represents the magic value in the optional header of the PE file.
- optionalmajor_image_version: This feature represents the major image version specified in the optional header of the PE file.
- optionalminor_image_version: This feature represents the minor image version specified in the optional header of the PE file.
- optionalmajor_linker_version: This feature represents the major linker version specified in the optional header of the PE file.
- optionalminor_linker_version: This feature represents the minor linker version specified in the optional header of the PE file.
- optionalmajor_operating_system_version: This feature represents the major operating system version specified in the optional header of the PE file.
- optionalminor_operating_system_version: This feature represents the minor operating system version specified in the optional header of the PE file.
- optionalmajor_subsystem_version: This feature represents the major subsystem version specified in the optional header of the PE file.
- optionalminor_subsystem_version: This feature represents the minor subsystem version specified in the optional header of the PE file.
- optionalsizeof_code: This feature represents the size of the code section specified in the optional header of the PE file.
- optionalsizeof_headers: This feature represents the size of the headers specified in the optional header of the PE file.
- optionalsizeof_heap_commit: This feature represents the size of the committed heap specified in the optional header of the PE file.


In [ ]:
def load_dataset_from_jsonl(file_path):
    dataset = []
    
    for i in range(1,3):
        path= file_path + str(i) + '.jsonl'
        with open(path, 'r') as file:
            for line in file:
                data = json.loads(line)
                dataset.append(data)
    return dataset

file_path = 'ember/train_features_'

dataset = load_dataset_from_jsonl(file_path)



In [1]:

features= {}
features['label']=[]

def extract_features(feature, feature_name):
    global features
    
    for key, value in feature.items(): 
        
        string_name = feature_name + '' + key
        if string_name not in features:
            features[string_name]= []
        
        if isinstance(value, int) or isinstance(value, float):
            features[string_name].append(value)
        else:
            features[string_name].append(None)
    

section_names = ['.text', '.data', '.rsrc']
feature_names = ['size', 'entropy', 'vsize']

for section in section_names:
    for feature in feature_names:
        features[section + feature] = []

features['machine']=[]
features['subsystem']=[]


for data in dataset:
    
    if data['label'] == -1:
        continue

#     val1, val2, val3=0,0,0
    section_count_list = {'.text':0, '.data':0, '.rsrc':0}
    for item in data['section']['sections']:
        
        if item['name'] in section_names:
            section_count_list[item['name']]+=1
    
    check = False
    for key,value in section_count_list.items():
        if value!=1:
            check=True
            break
    
    if check:
        continue
        
    feature_names = ['size', 'entropy', 'vsize']
        
    for item in data['section']['sections']:

        if item['name'] in section_names:
            
            for feature in feature_names:
                val = item[feature] if item[feature]!=None else None
                features[item['name'] + feature].append(val)
    
    feature_names = ['general', 'strings']

    for feature in feature_names:
        extract_features(data[feature], feature)

    for item in data['header']:
        extract_features(data['header'][item], item)
        
    features['machine'].append(data['header']['coff']['machine'])
    features['subsystem'].append(data['header']['optional']['subsystem'])
    
    
    features['label'].append(data['label'])

df = pd.DataFrame(features)
print(df['label'].value_counts())


NameError: name 'dataset' is not defined

In [ ]:
print(df.isnull().any())

In [ ]:
df.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(df.info())

In [ ]:

columns_to_encode = ['machine', 'subsystem']

for column in columns_to_encode:

    encoder = LabelEncoder()
    df[column] = encoder.fit_transform(df[column])

print(df.info())

In [ ]:
print(df.var())

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
for column in df.columns:
    sns.boxplot(df[column])
    plt.title(f'Distribution of {column}')
    plt.show()

In [ ]:
# Columns to exclude from standardization
exclude_columns= [
"machine", "subsystem", "label"
]

df_standardized = df.copy()

scaler = StandardScaler()

for column in df.columns:
    if column not in exclude_columns:
        # Standardize the column
        df_standardized[column] = scaler.fit_transform(df[column].values.reshape(-1, 1))

print(df_standardized)

In [ ]:
# df_standardized.to_csv('ember_data.csv')

In [ ]:
print(df_standardized['label'].nunique())

In [ ]:
# Get the count of each unique value
value_counts = y.value_counts()

# Retrieve the count of the specific value
c1 = value_counts.get(1, 0)
c2 = value_counts.get(0, 0)

# Print the result
print("Count of", 1, "in column:", c1, c2)

In [ ]:
y = df['label']

X = df.drop('label', axis=1)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
rf = RandomForestClassifier()

rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)